In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
from nltk.corpus import stopwords

In [9]:
blog_data = pd.read_csv('blogtext.csv')[:100000]

In [60]:
blog_data.drop(columns=['id','date'],inplace=True)

In [64]:
blog_data=blog_data.astype({'age':str})

In [65]:
blog_data.head()

,gender,age,topic,sign,text
0,male,15,Student,Leo,info found pages mb pdf files wait untill team...
1,male,15,Student,Leo,team members drewes van der laag urllink mail ...
2,male,15,Student,Leo,het kader van kernfusie op aarde maak je eigen...
3,male,15,Student,Leo,testing testing
4,male,33,InvestmentBanking,Aquarius,thanks yahoo toolbar capture urls popups means...


In [46]:
label_dict={}
for each_gender in blog_data['gender'].unique():
    label_dict[each_gender]=blog_data[blog_data['gender']==each_gender]['gender'].count()
for each_age in blog_data['age'].unique():
    label_dict[each_age]=blog_data[blog_data['age']==each_age]['age'].count()
for each_topic in blog_data['topic'].unique():
    label_dict[each_topic]=blog_data[blog_data['topic']==each_topic]['topic'].count()
for each_sign in blog_data['sign'].unique():
    label_dict[each_sign]=blog_data[blog_data['sign']==each_sign]['sign'].count()

In [8]:
def clean_text(text):
    text = BeautifulSoup(text).get_text()
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower().strip()
    words=text.split()
    words=[word for word in words if not word in stopwords.words('english')]
    return (" ".join(words))

In [5]:
text_column = blog_data['text']
blog_data.drop(columns='text',inplace=True)
text = []
for each_text in tqdm(text_column):
    text.append(clean_text(each_text))
blog_data.insert(column='text',loc=len(blog_data.columns),value=text)

 37%|███████████▌                   | 37474/100000 [1:12:33<3:02:52,  5.70it/s]C:\Users\tshrgpt329\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'           . .... ...         '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
 40%|████████████▍                  | 40310/100000 [1:17:54<2:44:37,  6.04it/s]C:\Users\tshrgpt329\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'                 .      '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
 45%|██████████████▋                  | 44603/100000 [1:26:30<27:20, 33.76it/s]C:\Users\tshrgpt329\Anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'       .         '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' 

In [66]:
labels=[]
for i in tqdm(blog_data[['gender','age','topic','sign']].values):
    labels.append(i)
blog_data.insert(column='labels',loc=len(blog_data.columns),value=labels)
blog_data.drop(columns=['gender','age','topic','sign'],inplace=True)

100%|██████████████████████████████| 100000/100000 [00:00<00:00, 406179.60it/s]


In [67]:
blog_data = blog_data.dropna()
blog_data.shape

(99497, 2)

In [68]:
target = blog_data['labels']
blog_data.drop(columns='labels',inplace=True)

In [69]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(blog_data,target,train_size=0.8,shuffle=True)

In [70]:
train_x_array = []
test_x_array = []
for each_train_x in train_x['text']:
    train_x_array.append(each_train_x)
for each_train_y in test_x['text']:
    test_x_array.append(each_train_y)

In [71]:
train_x_array[9]

'w ell bear one think might starting sect err school thought prefer entrepreneurial title consultant firm think tank think tank seems bit pretentious besides one needs except really really big wigs right small potatos sweet job would though full time member company think tank sit around bunch smart people try come solutions many types problems company may run unpredictable unseen otherwise novel problems course lucrative financially present worthwhile opportunity participate one favorite mental sports time solving problems via encorporation knowledge intellect solid grasp logic solutions problems would definitely worked efficiently quickly convincingly enterprise worked collective think know talking clever enough pool resources together ultimately produce satistying results previously demonstrated many times past philosophy schoolwork jamming btf etc hey serious whatever clientele well hey would job would could enjoy really know guys wanna job hate wake every morning know miserable nex

In [72]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500,
                            ngram_range = (1,2)) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(train_x_array)
test_data_features = vectorizer.fit_transform(test_x_array)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

In [73]:
train_data_features[0]

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [74]:
label_dict

{'male': 53358,
 'female': 46642,
 15: 6532,
 33: 2835,
 14: 3540,
 25: 8660,
 17: 12755,
 23: 10757,
 37: 863,
 26: 8059,
 24: 11814,
 27: 8007,
 45: 906,
 34: 2388,
 41: 772,
 44: 76,
 16: 8406,
 39: 568,
 35: 4720,
 36: 3045,
 46: 914,
 42: 156,
 13: 1497,
 38: 801,
 43: 505,
 40: 513,
 47: 397,
 48: 514,
 'Student': 22122,
 'InvestmentBanking': 244,
 'indUnk': 33097,
 'Non-Profit': 1326,
 'Banking': 354,
 'Education': 5553,
 'Engineering': 2332,
 'Science': 1090,
 'Communications-Media': 2830,
 'BusinessServices': 626,
 'Sports-Recreation': 406,
 'Arts': 5031,
 'Internet': 2251,
 'Museums-Libraries': 308,
 'Accounting': 528,
 'Technology': 8484,
 'Law': 360,
 'Consulting': 905,
 'Automotive': 124,
 'Religion': 1081,
 'Fashion': 1898,
 'Publishing': 1079,
 'Marketing': 726,
 'LawEnforcement-Security': 368,
 'HumanResources': 209,
 'Telecommunications': 165,
 'Military': 798,
 'Government': 2055,
 'Transportation': 745,
 'Architecture': 83,
 'Advertising': 766,
 'Agriculture': 168,
 

In [79]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer()
train_y=mb.fit_transform(train_y)

In [80]:
test_y=mb.fit_transform(test_y)

In [77]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

In [84]:
from sklearn.metrics import precision_score,f1_score,average_precision_score
clf.fit(train_data_features,train_y)
print('precision {}'.format(precision_score(y_true=test_y,y_pred = clf.predict(test_data_features),average='weighted')))
print('F1 score {}'.format(f1_score(y_true=test_y,y_pred = clf.predict(test_data_features),average='weighted')))
      

C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of

C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\tshrgpt329\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of

precision 0.2412463117125545
F1 score 0.1415040517056314
